In [18]:
#cells will fill entire width of the browser
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

#Tells Jupyter to reload custom classes from scratch everytime an import cell is run, if you edit a custom class
#between imports Jupyter would otherwise need to be restarted completely. Buyer beware: old class objects in the 
#current namespace will cause errors at execution
%load_ext autoreload
%autoreload 2

#switches matplotlib to show plots in the browser rather than opening a new window
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import xml.etree.ElementTree as ET
import numpy as np
import os
import sys
import shutil
from sklearn.model_selection import ParameterGrid
import multiprocessing
import random

#### global simulation/read-write parameters

give write_directory new name for new experiments, and adjust fixed/parking variables as seen fit

In [20]:
#get cores per process, can do 4 processes with standalone license

cores_per_process = int(multiprocessing.cpu_count()/4) #2 simulations in parallel for 8 cores, max 4 simulations on simulateously
use_all_cores = "false"
number_of_runs_per_exp = 1
sim_period = 9000
sim_resolution = 10 #measurements per second
# ^ I changed this from 10 10/13/2022!!!

time_steps_per_vehicle_measurement = 10 #this means vehcile measurements per unit simulation resolution
# ^ I changed this from 10 10/13/2022!!!

# write_directory = "E:\\20220511_experiments_multi_block\\" #name set manually
write_directory = "E:\\20221211_experiments_Cruise1\\" #name set manually 
#subdirectory
laneconfig = "2lane"


## Scenario parameter sweep values

#### Fixed values and parking variables

In [21]:
#check fixed for all sim driving behavior
jerkLimit="false"
consNextTurn="true"
recovSlow="false"
recovDist="1000"
distractProb="0.01"

#variable across experiments
parking_rates = [0.05, 0.10, 0.15, 0.20, 0.25, 0.3]
speed_dists = [25]

In [22]:
#adjust mean and stdDEv per distribution number, let distribution numbers be fixed for vehicleRoutingDecisionsParking
#so each of these just maps to vehicle class, only need to worry about editting 50, 60, 70, and 80

#for no 50
bus_means = [90] #in seconds
bus_stds = [0.25*i for i in bus_means] #each of these maps to each of the means
bus_park_dists = zip(bus_means, bus_stds)

#for no 60
car_means = [600]
car_stds = [0.25*i for i in car_means] #each of these maps to each of the means
car_park_dists = zip(car_means, car_stds)

#for no 70
tnc_means = [30]
tnc_stds = [0.25*i for i in tnc_means]
tnc_park_dists = zip(tnc_means, tnc_stds)

#HGV_dwell
hgv_means = [300, 850]
hgv_stds = [0.25*i for i in hgv_means]
hgv_park_dists = zip(hgv_means, hgv_stds)

#volumes
#volumes = np.arange(150,850,100) 

#compositions
#TNCcomp = np.arange(0.20,0.3,0.05)
#CVcomp = np.arange(0.115,0.155,0.02)
#PAXcomp = 0.935-TNCcomp-CVcomp
#BUScomp = 0.03 #np.arange(0.03,0.03,1)
#HGVcomp = 0.035 #np.arange(0.035,0.035,1)

#### Vehicle volumes and compositions

In [23]:
#volumes = np.arange(0,500,100) #vehicles per hour
volumes = np.arange(50,450,50) 
#super duper fancy uniform sampling from n-dim simplex Donald B. Rubin, The Bayesian bootstrap Ann. Statist. 9, 1981, 130-134.
MC_size = 0
num_vehicle_types = 5
#new distributions for TNC, CV fleet composition share
#TNCcomp = np.arange(0.20,0.3,0.05)
#CVcomp = np.arange(0.115,0.155,0.02)
#PAXcomp = 0.935-TNCcomp-CVcomp
#BUScomp = np.arange(0.03,0.03,1)
#HGVcomp = np.arange(0.035,0.035,1)
vehicle_ratios_MC = [[0.55, 0.03, 0.035, 0.25, 0.135]] #initialize with a default sample, each decimal refers to a vehicle proporiton
#vehicle_ratios_MC = [[PAXcomp, 0.03, 0.035, TNCcomp, CVcomp]]

for i in range(MC_size):
    #take N-1 samples from Unif(0,1), sort, compute first difference on [0, x_1, x_2, x_n-1, 1]
    pre_ratios = [0, 1]
    for n in range(num_vehicle_types-1):
        pre_ratios.append(np.random.uniform(0,1))

    sorted_pre_ratios = sorted(pre_ratios)
    veh_ratios = list(np.diff(pre_ratios))

    vehicle_ratios_MC.append(veh_ratios)
    
#ensure each vertex of the simplex is present in the sample
#for i in range(num_vehicle_types):
#    temp_ratios = [0 for j in range(num_vehicle_types)]
#    temp_ratios[i] = 1.0
#    vehicle_ratios_MC.append(temp_ratios)

In [24]:
param_grid = {
              'bus_park_dists': bus_park_dists,
              'car_park_dists': car_park_dists,
              'tnc_park_dists': tnc_park_dists,
              'hgv_park_dists': hgv_park_dists,
              'vehicleVolumes': volumes,
              'speed_dists': speed_dists,
              'vehicleCompositions': vehicle_ratios_MC,
              'parking_rates': parking_rates
             }

param_sweep = list(ParameterGrid(param_grid)) #each list item is a dict with a unique key-value combination of the above
                                              #key-list pairs

In [25]:
len(param_sweep)

96

## Populate experiment output directories

#### Read in scenario baseline inpx files

In [26]:
base_path = "C:\\Users\\maxn363\\Documents\\vissimpark\\" 
scenario_dirs = os.listdir(base_path + "VISSIM_configs\\atomic_network_multi_block\\" + laneconfig + "\\")


for dirname in scenario_dirs:
    #get baseline scenario directory path, inpx path
    scenario_path = base_path + "VISSIM_configs\\atomic_network_multi_block\\" + laneconfig + "\\" + dirname + "\\"
    inpx_file_path =  scenario_path + "Seattle_Atomic_Network_" + dirname + ".inpx" #this is the master simulation configuration file
    
    copy_files = ["SLU-v21.sig", "Seattle_Atomic_network_" + dirname + ".layx"]
    
    #create experiment output directory
    if not os.path.exists(write_directory):
        os.mkdir(write_directory)
    
    outdir = write_directory + "\\" + dirname
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    #create experiment registry
    paramfile = open(outdir + "\\experiment_registry.txt", 'w')
    param_names = sorted(param_grid.keys()) 
    header = "experiment_number," + ",".join(param_names)
    paramfile.write("header\n")
    
    #create logfile to record errors, runtimes
    with open(outdir + "\\experiment_logfile.txt", 'w') as d:
        d.write("experiment_number,runtime,stdout\n")
    
    experiment_counter = 1
    for param_set in param_sweep:
        values = [ str(param_set[key]) for key in param_names ]
        paramfile.write(str(experiment_counter) + "," + ",".join(values) + "\n")
        
        #write experiment directory
        exp_dir = outdir + "\\experiment_" + str(experiment_counter)
        if not os.path.exists(exp_dir):
            os.mkdir(exp_dir)
            
        #copy sig files, layx file
        for file_name in copy_files:
            full_file_name = os.path.join(scenario_path, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, exp_dir)
        
        #read, edit, write inpx file
        tree = ET.parse(inpx_file_path)
        root = tree.getroot()
        
        ###ensure fixed values are correct
        #driving behaviors
        for v in root.iter('drivingBehaviors'):
            v.set('jerkLimit', 'false')
            v.set('consNextTurn', 'true')
            v.set('recovSlow', 'false')
            v.set('recovDist', '1000')
            v.set('distractProb', '0.01')
            
        for v in root.iter('simulation'):
            v.set('numCores', str(cores_per_process))
            v.set('numRuns', str(number_of_runs_per_exp))
            v.set('useAllCores', use_all_cores)
            v.set('simPeriod', str(sim_period))
            v.set('randSeed', str(random.randrange(1,5000)))
            v.set('simRes', str(sim_resolution))
            
        for v in root.iter('vehRec'):
            v.set('resolution', str(time_steps_per_vehicle_measurement))
            
        #set warmup period for spitting out collected data
        #ignoring vehicles already in network given toggled setting
        warmup = '1800' #this should probably get moved up
        for v in root.iter('dataColl'):
            v.set('fromTime', warmup)
            
        for v in root.iter('dataCollRawData'):
            v.set('fromtTime', warmup)
            
        for v in root.iter('vehInps'):
            v.set('fromtTime', warmup)
            
        for v in root.iter('vehRec'):
            v.set('fromtTime', warmup)
        
        ###edit variable values
        #vehicle volume
        vol = param_set['vehicleVolumes']
        for v in root.iter('timeIntervalVehVolume'):
            v.set('volume', str(vol))
            
        ###speed dist and vehicle composition
        veh_type_counter = 0
        vehicle_comps = param_set['vehicleCompositions']
        for v in root.iter('vehCompRelFlows'):
            v.set('desSpeedDistr', str(param_set['speed_dists']))
            v.set('relFlow', str(vehicle_comps[veh_type_counter]))
            veh_type_counter += 1
            
        ###parking times
        for v in root.iter('timeDistribution'):
            #cars
            if v.attrib['no'] == '60':
                v.set('mean', str(param_set['car_park_dists'][0]))
                v.set('stdDev', str(param_set['car_park_dists'][1]))
            #busses
            if v.attrib['no'] == '50':
                v.set('mean', str(param_set['bus_park_dists'][0]))
                v.set('stdDev', str(param_set['bus_park_dists'][1]))
            #tncs
            if v.attrib['no'] == '70':
                v.set('mean', str(param_set['tnc_park_dists'][0]))
                v.set('stdDev', str(param_set['tnc_park_dists'][1]))
            #hgv's
            if v.attrib['no'] == '80':
                v.set('mean', str(param_set['hgv_park_dists'][0]))
                v.set('stdDev', str(param_set['hgv_park_dists'][1]))
                
        ###parking rates
        for v in root.iter('timeIntParkRate'):
            v.set('parkRate', str(param_set['parking_rates']))
        
                
        #write final inpx file
        tree.write(exp_dir + "\\Seattle_Atomic_Network_" + dirname + "_experiment_" + str(experiment_counter) + ".inpx")
        
        experiment_counter += 1
        
    paramfile.close()
    

In [27]:
#complete


#### global simulation/read-write parameters

give write_directory new name for new experiments, and adjust fixed/parking variables as seen fit

In [28]:
#get cores per process, can do 4 processes with standalone license

cores_per_process = int(multiprocessing.cpu_count()/4) #2 simulations in parallel for 8 cores, max 4 simulations on simulateously
use_all_cores = "false"
number_of_runs_per_exp = 1
sim_period = 9000
sim_resolution = 10 #measurements per second
# ^ I changed this from 10 10/13/2022!!!

time_steps_per_vehicle_measurement = 10 #this means vehcile measurements per unit simulation resolution
# ^ I changed this from 10 10/13/2022!!!

# write_directory = "E:\\20220511_experiments_multi_block\\" #name set manually
write_directory = "E:\\20221211_experiments_Cruise2\\" #name set manually 
#subdirectory
laneconfig = "Llane"


## Scenario parameter sweep values

#### Fixed values and parking variables

In [29]:
#check fixed for all sim driving behavior
jerkLimit="false"
consNextTurn="true"
recovSlow="false"
recovDist="1000"
distractProb="0.01"

#variable across experiments
parking_rates = [0.05, 0.10, 0.15, 0.20, 0.25, 0.3]
speed_dists = [25]

In [30]:
#adjust mean and stdDEv per distribution number, let distribution numbers be fixed for vehicleRoutingDecisionsParking
#so each of these just maps to vehicle class, only need to worry about editting 50, 60, 70, and 80

#for no 50
bus_means = [90] #in seconds
bus_stds = [0.25*i for i in bus_means] #each of these maps to each of the means
bus_park_dists = zip(bus_means, bus_stds)

#for no 60
car_means = [600]
car_stds = [0.25*i for i in car_means] #each of these maps to each of the means
car_park_dists = zip(car_means, car_stds)

#for no 70
tnc_means = [30]
tnc_stds = [0.25*i for i in tnc_means]
tnc_park_dists = zip(tnc_means, tnc_stds)

#HGV_dwell
hgv_means = [300, 850]
hgv_stds = [0.25*i for i in hgv_means]
hgv_park_dists = zip(hgv_means, hgv_stds)

#volumes
#volumes = np.arange(150,850,100) 

#compositions
#TNCcomp = np.arange(0.20,0.3,0.05)
#CVcomp = np.arange(0.115,0.155,0.02)
#PAXcomp = 0.935-TNCcomp-CVcomp
#BUScomp = 0.03 #np.arange(0.03,0.03,1)
#HGVcomp = 0.035 #np.arange(0.035,0.035,1)

#### Vehicle volumes and compositions

In [31]:
#volumes = np.arange(0,500,100) #vehicles per hour
volumes = np.arange(50,450,50) 
#super duper fancy uniform sampling from n-dim simplex Donald B. Rubin, The Bayesian bootstrap Ann. Statist. 9, 1981, 130-134.
MC_size = 0
num_vehicle_types = 5
#new distributions for TNC, CV fleet composition share
#TNCcomp = np.arange(0.20,0.3,0.05)
#CVcomp = np.arange(0.115,0.155,0.02)
#PAXcomp = 0.935-TNCcomp-CVcomp
#BUScomp = np.arange(0.03,0.03,1)
#HGVcomp = np.arange(0.035,0.035,1)
vehicle_ratios_MC = [[0.55, 0.03, 0.035, 0.25, 0.135]] #initialize with a default sample, each decimal refers to a vehicle proporiton
#vehicle_ratios_MC = [[PAXcomp, 0.03, 0.035, TNCcomp, CVcomp]]

for i in range(MC_size):
    #take N-1 samples from Unif(0,1), sort, compute first difference on [0, x_1, x_2, x_n-1, 1]
    pre_ratios = [0, 1]
    for n in range(num_vehicle_types-1):
        pre_ratios.append(np.random.uniform(0,1))

    sorted_pre_ratios = sorted(pre_ratios)
    veh_ratios = list(np.diff(pre_ratios))

    vehicle_ratios_MC.append(veh_ratios)
    
#ensure each vertex of the simplex is present in the sample
#for i in range(num_vehicle_types):
#    temp_ratios = [0 for j in range(num_vehicle_types)]
#    temp_ratios[i] = 1.0
#    vehicle_ratios_MC.append(temp_ratios)

In [32]:
param_grid = {
              'bus_park_dists': bus_park_dists,
              'car_park_dists': car_park_dists,
              'tnc_park_dists': tnc_park_dists,
              'hgv_park_dists': hgv_park_dists,
              'vehicleVolumes': volumes,
              'speed_dists': speed_dists,
              'vehicleCompositions': vehicle_ratios_MC,
              'parking_rates': parking_rates
             }

param_sweep = list(ParameterGrid(param_grid)) #each list item is a dict with a unique key-value combination of the above
                                              #key-list pairs

In [33]:
len(param_sweep)

96

## Populate experiment output directories

#### Read in scenario baseline inpx files

In [34]:
base_path = "C:\\Users\\maxn363\\Documents\\vissimpark\\" 
scenario_dirs = os.listdir(base_path + "VISSIM_configs\\atomic_network_multi_block\\" + laneconfig + "\\")


for dirname in scenario_dirs:
    #get baseline scenario directory path, inpx path
    scenario_path = base_path + "VISSIM_configs\\atomic_network_multi_block\\" + laneconfig + "\\" + dirname + "\\"
    inpx_file_path =  scenario_path + "Seattle_Atomic_Network_" + dirname + ".inpx" #this is the master simulation configuration file
    
    copy_files = ["SLU-v21.sig", "Seattle_Atomic_network_" + dirname + ".layx"]
    
    #create experiment output directory
    if not os.path.exists(write_directory):
        os.mkdir(write_directory)
    
    outdir = write_directory + "\\" + dirname
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    #create experiment registry
    paramfile = open(outdir + "\\experiment_registry.txt", 'w')
    param_names = sorted(param_grid.keys()) 
    header = "experiment_number," + ",".join(param_names)
    paramfile.write("header\n")
    
    #create logfile to record errors, runtimes
    with open(outdir + "\\experiment_logfile.txt", 'w') as d:
        d.write("experiment_number,runtime,stdout\n")
    
    experiment_counter = 1
    for param_set in param_sweep:
        values = [ str(param_set[key]) for key in param_names ]
        paramfile.write(str(experiment_counter) + "," + ",".join(values) + "\n")
        
        #write experiment directory
        exp_dir = outdir + "\\experiment_" + str(experiment_counter)
        if not os.path.exists(exp_dir):
            os.mkdir(exp_dir)
            
        #copy sig files, layx file
        for file_name in copy_files:
            full_file_name = os.path.join(scenario_path, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, exp_dir)
        
        #read, edit, write inpx file
        tree = ET.parse(inpx_file_path)
        root = tree.getroot()
        
        ###ensure fixed values are correct
        #driving behaviors
        for v in root.iter('drivingBehaviors'):
            v.set('jerkLimit', 'false')
            v.set('consNextTurn', 'true')
            v.set('recovSlow', 'false')
            v.set('recovDist', '1000')
            v.set('distractProb', '0.01')
            
        for v in root.iter('simulation'):
            v.set('numCores', str(cores_per_process))
            v.set('numRuns', str(number_of_runs_per_exp))
            v.set('useAllCores', use_all_cores)
            v.set('simPeriod', str(sim_period))
            v.set('randSeed', str(random.randrange(1,5000)))
            v.set('simRes', str(sim_resolution))
            
        for v in root.iter('vehRec'):
            v.set('resolution', str(time_steps_per_vehicle_measurement))
            
        #set warmup period for spitting out collected data
        #ignoring vehicles already in network given toggled setting
        warmup = '1800' #this should probably get moved up
        for v in root.iter('dataColl'):
            v.set('fromTime', warmup)
            
        for v in root.iter('dataCollRawData'):
            v.set('fromtTime', warmup)
            
        for v in root.iter('vehInps'):
            v.set('fromtTime', warmup)
            
        for v in root.iter('vehRec'):
            v.set('fromtTime', warmup)
        
        ###edit variable values
        #vehicle volume
        vol = param_set['vehicleVolumes']
        for v in root.iter('timeIntervalVehVolume'):
            v.set('volume', str(vol))
            
        ###speed dist and vehicle composition
        veh_type_counter = 0
        vehicle_comps = param_set['vehicleCompositions']
        for v in root.iter('vehCompRelFlows'):
            v.set('desSpeedDistr', str(param_set['speed_dists']))
            v.set('relFlow', str(vehicle_comps[veh_type_counter]))
            veh_type_counter += 1
            
        ###parking times
        for v in root.iter('timeDistribution'):
            #cars
            if v.attrib['no'] == '60':
                v.set('mean', str(param_set['car_park_dists'][0]))
                v.set('stdDev', str(param_set['car_park_dists'][1]))
            #busses
            if v.attrib['no'] == '50':
                v.set('mean', str(param_set['bus_park_dists'][0]))
                v.set('stdDev', str(param_set['bus_park_dists'][1]))
            #tncs
            if v.attrib['no'] == '70':
                v.set('mean', str(param_set['tnc_park_dists'][0]))
                v.set('stdDev', str(param_set['tnc_park_dists'][1]))
            #hgv's
            if v.attrib['no'] == '80':
                v.set('mean', str(param_set['hgv_park_dists'][0]))
                v.set('stdDev', str(param_set['hgv_park_dists'][1]))
                
        ###parking rates
        for v in root.iter('timeIntParkRate'):
            v.set('parkRate', str(param_set['parking_rates']))
        
                
        #write final inpx file
        tree.write(exp_dir + "\\Seattle_Atomic_Network_" + dirname + "_experiment_" + str(experiment_counter) + ".inpx")
        
        experiment_counter += 1
        
    paramfile.close()
    

In [35]:
#complete
